In [4]:
# !pip install numpy, tensorflow, keras, pillow,

In [9]:
# !pip install "numpy>=1.16.5,<1.23.0"

In [8]:
# 1. Import the libraries and load the dataset

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [15]:
# 2. Preprocess the data

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes=None)
y_test = keras.utils.to_categorical(y_test, num_classes=None)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [17]:
# 3. Create the model


batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [27]:
# 4. Train the model

# hist = model.fit(x_train, y_train, epochs=3)
hist = model.fit(x_train, 
                 y_train, batch_size = batch_size,epochs = epochs,
                 verbose = 1, validation_data =(x_test, y_test))
print("The model has successfully trained")

# model.save('mnist.h5')
model.save('mnist.model')
print("Saving the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 1506s 3s/step - loss: 0.6237 - accuracy: 0.8089 - val_loss: 0.4589 - val_accuracy: 0.8778
Epoch 2/10
469/469 [==============================] - 522s 1s/step - loss: 0.6026 - accuracy: 0.8162 - val_loss: 0.4417 - val_accuracy: 0.8818
Epoch 3/10
469/469 [==============================] - 668s 1s/step - loss: 0.5837 - accuracy: 0.8202 - val_loss: 0.4274 - val_accuracy: 0.8849
Epoch 4/10
469/469 [==============================] - 969s 2s/step - loss: 0.5661 - accuracy: 0.8266 - val_loss: 0.4141 - val_accuracy: 0.8886
Epoch 5/10
469/469 [==============================] - 578s 1s/step - loss: 0.5545 - accuracy: 0.8288 - val_loss: 0.4026 - val_accuracy: 0.8913
Epoch 6/10
469/469 [==============================] - 973s 2s/step - loss: 0.5429 - accuracy: 0.8329 - val_loss: 0.3931 - val_accuracy: 0.8939
Epoch 7/10
469/469 [==============================] - 946s 2s/step - loss: 0.5284 - accuracy: 0.8386 - val_loss: 0.3838 - val_accuracy: 0.895

FailedPreconditionError: mnist.model\variables is not a directory

In [25]:
# 5. Evaluate the model

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4774414896965027
Test accuracy: 0.8758000135421753


In [26]:
# 6. Create GUI to predict digits

model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 128ms/step
